In [109]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass

In [110]:
STRATEGIES = ['Venture Capital', 'Growth Equity', 'Private Equity']
MONETARY_COLS = ['Capital Committed', 'Capital Contributed', 'Capital Distributed', 'Market Value']
CALSTRS_PATH = 'CalSTRSPrivateEquityPerformanceReportFYE20241.csv'

In [150]:
@dataclass
class PlotStyle:
    COLORS = {
        'Venture Capital': 'rgb(30,144,255)',  # Blue
        'Growth Equity': 'rgb(60,179,113)',    # Green
        'Private Equity': 'rgb(200,30,30)'     # Red
    }
    FONT_FAMILY = "Avenir"
    TITLE_SIZE = 25
    AXIS_TITLE_SIZE = 18
    AXIS_TICK_SIZE = 16
    LEGEND_SIZE = 14
    ANNOTATION_SIZE = 14
    
    @classmethod
    def get_base_layout(cls, height: int = 800, title: str = "") -> dict:
        return {
            'height': height,
            'title': {
                'text': f"<b>{title}</b>",
                'y': 0.95,
                'x': 0.5,
                'xanchor': 'center',
                'yanchor': 'top'
            },
            'title_font': dict(family=cls.FONT_FAMILY, size=cls.TITLE_SIZE),
            'margin': dict(b=50, l=50, r=50, t=100),
            'plot_bgcolor': 'white',
            'showlegend': True,
            'legend': dict(
                orientation="h",
                yanchor="bottom",
                y=-0.15,
                xanchor="center",
                x=0.5,
                font=dict(family=cls.FONT_FAMILY, size=cls.LEGEND_SIZE)
            )
        }
    
    @staticmethod
    def calculate_tick_values(max_value: float) -> Tuple[np.ndarray, List[str]]:
        tick_step = 0.5  # 0.5B intervals
        max_billions = np.ceil(max_value / 1e9)
        tick_values = np.arange(0, tick_step * (int(max_billions/tick_step) + 2), tick_step) * 1e9
        tick_labels = [f"${val/1e9:.1f}B" if val > 0 else "0" for val in tick_values]
        return tick_values, tick_labels

class DataProcessor:
    @staticmethod
    def clean_data(df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        
        for col in MONETARY_COLS:
            df[col] = df[col].replace('[\$,]', '', regex=True).replace('-', '0').astype(float)
        
        df['CalSTRS Since Inception IRR*'] = pd.to_numeric(df['CalSTRS Since Inception IRR*'], errors='coerce')
        df['VY'] = pd.to_numeric(df['VY'], errors='coerce')
        df['DPI'] = df['Capital Distributed'] / df['Capital Contributed']
        
        return df
    
    @staticmethod
    def calculate_weighted_irr(group: pd.DataFrame) -> float:
        total_capital = group['Capital Contributed'].sum()
        if total_capital == 0:
            return 0
        weights = group['Capital Contributed'] / total_capital
        return (group['CalSTRS Since Inception IRR*'] * weights).sum()

class VisualizationGenerator:    
    @staticmethod
    def create_venture_capital_deployment_chart(df: pd.DataFrame) -> go.Figure:
        vc_by_year = df[df['Fund Type'] == 'Venture Capital'].groupby('VY')['Capital Contributed'].sum()
        total_by_year = df.groupby('VY')['Capital Contributed'].sum()
        vc_percentage = (vc_by_year / total_by_year * 100).fillna(0)
        
        all_years = pd.Index(range(df['VY'].min(), df['VY'].max() + 1))
        vc_by_year = vc_by_year.reindex(all_years, fill_value=0)
        vc_percentage = vc_percentage.reindex(all_years, fill_value=0)
        
        fig = go.Figure()
        
        fig.add_trace(
            go.Bar(
                x=vc_by_year.index,
                y=vc_by_year.values,
                name="VC Deployment",
                text=[f"${val/1e6:.0f}M" for val in vc_by_year.values],
                textposition='outside',
                marker_color=PlotStyle.COLORS['Venture Capital'],
                hovertemplate=(
                    "Year: %{x}<br>"
                    "Amount: $%{y:,.0f}<br>"
                    "<extra></extra>"
                )
            )
        )
        
        fig.add_trace(
            go.Scatter(
                x=vc_percentage.index,
                y=vc_percentage.values,
                name="% of Total PE Deployment",
                text=[f"{val:.1f}%" for val in vc_percentage.values],
                textposition='top center',
                mode='lines+markers+text',
                line=dict(color='rgb(60,179,113)', width=2),
                marker=dict(size=8),
                yaxis='y2',
                hovertemplate=(
                    "Year: %{x}<br>"
                    "Percentage: %{y:.1f}%<br>"
                    "<extra></extra>"
                )
            )
        )
        
        layout = PlotStyle.get_base_layout(title="Venture Capital Deployment by Year")
        layout.update({
            'xaxis': {
                'dtick': 1,
                'gridcolor': 'lightgrey'
            },
            'yaxis': {
                'gridcolor': 'lightgrey',
                'tickmode': 'array',
            },
            'yaxis2': {
                'title': "Percentage of Total Deployment",
                'titlefont': dict(family=PlotStyle.FONT_FAMILY, size=PlotStyle.AXIS_TITLE_SIZE),
                'tickfont': dict(family=PlotStyle.FONT_FAMILY, size=PlotStyle.AXIS_TICK_SIZE),
                'ticksuffix': '%',
                'overlaying': 'y',
                'side': 'right',
                'range': [0, 100]
            }
        })
        
        fig.update_layout(layout)
        return fig
    
    @staticmethod
    def create_strategy_pie_charts(df: pd.DataFrame) -> go.Figure:
        strategy_by_dollars = df.groupby('Fund Type')['Capital Contributed'].sum()
        strategy_by_count = df['Fund Type'].value_counts()
        
        total_capital = strategy_by_dollars.sum()
        total_funds = strategy_by_count.sum()
        
        fig = make_subplots(
            rows=1, cols=2,
            specs=[[{'type':'domain'}, {'type':'domain'}]],
            horizontal_spacing=0.1,
            subplot_titles=('', '')
        )
        
        capital_text = [
            f"{strat}<br>${val/1e9:.1f}B / {val/total_capital*100:.1f}%"
            for strat, val in strategy_by_dollars.items()
        ]
        
        fig.add_trace(
            go.Pie(
                labels=strategy_by_dollars.index,
                values=strategy_by_dollars,
                text=capital_text,
                textinfo='text',
                textposition='outside',
                hoverinfo='text',
                hole=0.5,
                domain={'x': [0, 0.45]},
                textfont=dict(family="Avenir", size=18)
            ),
            row=1, col=1
        )
        
        count_text = [
            f"{strat}<br>{int(val)} funds / {val/total_funds*100:.1f}%"
            for strat, val in strategy_by_count.items()
        ]
        
        fig.add_trace(
            go.Pie(
                labels=strategy_by_count.index,
                values=strategy_by_count,
                text=count_text,
                textinfo='text',
                textposition='outside',
                hoverinfo='text',
                hole=0.5,
                domain={'x': [0.55, 1]},
                textfont=dict(family="Avenir", size=18)
            ),
            row=1, col=2
        )
        
        fig.update_layout(
            height=1000,
            showlegend=True,
            title={
                'text': "<b>Fund Strategy Distribution Analysis</b>",
                'y': 0.95,
                'x': 0.5,
                'xanchor': 'center',
                'yanchor': 'top'
            },
            title_font=dict(size=25, family="Avenir"),
            annotations=[
                dict(
                    text=f"<b>By Capital Contributed (${total_capital/1e9:.1f}B)</b>",
                    x=0.15, y=0.5,
                    font_size=18,
                    showarrow=False,
                    xref='paper',
                    yref='paper'
                ),
                dict(
                    text=f"<b>By Number of Funds ({int(total_funds)} Funds)</b>",
                    x=0.87, y=0.5,
                    font_size=18,
                    showarrow=False,
                    xref='paper',
                    yref='paper'
                )
            ],
            legend=dict(
                orientation="h",
                yanchor="top",
                y=-0.25,
                xanchor="center",
                x=0.5
            ),
            margin=dict(b=100),
            font=dict(family="Avenir", size=18)
        )
        
        return fig
    
    @staticmethod
    def create_deployment_chart(df: pd.DataFrame) -> go.Figure:
        yearly_strategy = df.pivot_table(
            values='Capital Contributed',
            index='VY',
            columns='Fund Type',
            aggfunc='sum',
            fill_value=0
        )
        
        max_value = yearly_strategy.sum(axis=1).max()
        tick_values, tick_labels = PlotStyle.calculate_tick_values(max_value)
        
        fig = go.Figure()
        yearly_totals = yearly_strategy.sum(axis=1)
        
        for strategy in yearly_strategy.columns:
            fig.add_trace(
                go.Bar(
                    x=yearly_strategy.index,
                    y=yearly_strategy[strategy],
                    name=strategy,
                    hovertemplate=(
                        f"{strategy}<br>"
                        "Year: %{x}<br>"
                        "Amount: $%{y:,.0f}<br>"
                        "<extra></extra>"
                    )
                )
            )
        
        annotations = [
            dict(
                x=year,
                y=total,
                text=f"${total/1e9:.1f}B",
                showarrow=False,
                yshift=10,
                font=dict(
                    family=PlotStyle.FONT_FAMILY,
                    size=PlotStyle.ANNOTATION_SIZE
                )
            )
            for year, total in yearly_totals.items()
        ]
        
        layout = PlotStyle.get_base_layout(title="Capital Deployment by Year and Strategy")
        layout.update({
            'barmode': 'stack',
            'annotations': annotations,
            'xaxis': {
                'title': "Vintage Year",
                'title_font': dict(family=PlotStyle.FONT_FAMILY, size=PlotStyle.AXIS_TITLE_SIZE),
                'tickfont': dict(family=PlotStyle.FONT_FAMILY, size=PlotStyle.AXIS_TICK_SIZE),
                'dtick': 1,
                'gridcolor': 'lightgrey'
            },
            'yaxis': {
                'title': "Capital Contributed ($)",
                'title_font': dict(family=PlotStyle.FONT_FAMILY, size=PlotStyle.AXIS_TITLE_SIZE),
                'tickfont': dict(family=PlotStyle.FONT_FAMILY, size=PlotStyle.AXIS_TICK_SIZE),
                'gridcolor': 'lightgrey',
                'tickmode': 'array',
                'tickvals': tick_values,
                'ticktext': tick_labels
            }
        })
        
        fig.update_layout(layout)
        return fig

    @staticmethod
    def create_capital_flows_chart(df: pd.DataFrame) -> go.Figure:        
        yearly_flows = df.groupby('VY').agg({
            'Capital Contributed': 'sum',
            'Capital Distributed': 'sum',
            'Market Value': 'sum'
        }).fillna(0)
        
        yearly_flows['DPI'] = (yearly_flows['Capital Distributed'] / yearly_flows['Capital Contributed']).round(2)
        
        all_years = pd.Index(range(df['VY'].min(), df['VY'].max() + 1))
        yearly_flows = yearly_flows.reindex(all_years, fill_value=0)
        
        fig = go.Figure()
        
        fig.add_trace(
            go.Bar(
                x=yearly_flows.index,
                y=-yearly_flows['Capital Contributed'],
                name='Capital Contributed',
                marker_color='rgb(200,30,30)',
                text=[f"-${val/1e9:.2f}B" for val in yearly_flows['Capital Contributed']],
                textposition='outside',
                hovertemplate=(
                    "Year: %{x}<br>"
                    "Capital Contributed: $%{y:,.0f}<br>"
                    "<extra></extra>"
                )
            )
        )
        
        for name, color in [('Capital Distributed', 'rgb(30,144,255)'), 
                          ('Market Value', 'rgb(60,179,113)')]:
            fig.add_trace(
                go.Bar(
                    x=yearly_flows.index,
                    y=yearly_flows[name],
                    name=name,
                    marker_color=color,
                    text=[f"${val/1e9:.2f}B" for val in yearly_flows[name]],
                    hovertemplate=(
                        f"Year: %{{x}}<br>"
                        f"{name}: $%{{y:,.0f}}<br>"
                        "<extra></extra>"
                    )
                )
            )
        
        annotations = [
            dict(
                x=year,
                y=yearly_flows.loc[year, 'Capital Distributed'] + yearly_flows.loc[year, 'Market Value'],
                text=f"DPI: {yearly_flows.loc[year, 'DPI']:.2f}x",
                showarrow=False,
                yshift=25,
                font=dict(family=PlotStyle.FONT_FAMILY, size=PlotStyle.ANNOTATION_SIZE)
            )
            for year in yearly_flows.index
            if yearly_flows.loc[year, 'Capital Contributed'] > 0
        ]
        
        layout = PlotStyle.get_base_layout(title="Capital Flows by Vintage Year")
        layout.update({
            'xaxis': {
                'dtick': 1,
                'gridcolor': 'lightgrey'
            },
            'yaxis': {
                'gridcolor': 'lightgrey',
                'tickmode': 'array',
            },
            'barmode': 'relative',
            'annotations': annotations
        })
        
        fig.update_layout(layout)
        return fig
        
    @staticmethod
    def create_vintage_irr_chart(df: pd.DataFrame) -> go.Figure:
        vintage_irrs = []
        
        for year, year_data in df.groupby('VY'):
            weighted_irr = DataProcessor.calculate_weighted_irr(year_data)
            vintage_irrs.append({'Year': year, 'Weighted_IRR': weighted_irr})
        
        vintage_data = pd.DataFrame(vintage_irrs).set_index('Year')
        all_years = pd.Index(range(df['VY'].min(), df['VY'].max() + 1))
        vintage_data = vintage_data.reindex(all_years, fill_value=0)
        
        fig = go.Figure()
        fig.add_trace(
            go.Bar(
                x=vintage_data.index,
                y=vintage_data['Weighted_IRR'],
                name='Weighted IRR',
                marker_color='rgb(30,144,255)',
                text=[f"{val:.1f}%" for val in vintage_data['Weighted_IRR']],
                textposition='outside',
                hovertemplate=(
                    "Year: %{x}<br>"
                    "Weighted IRR: %{y:.1f}%<br>"
                    "<extra></extra>"
                )
            )
        )
        
        fig.add_hline(y=0, line_dash="dash", line_color="gray", line_width=1)
        
        layout = PlotStyle.get_base_layout(title="Weighted IRR by Vintage Year")
        layout.update({
            'showlegend': True,
            'xaxis': {
                'dtick': 1,
                'gridcolor': 'lightgrey'
            },
            'yaxis': {
                'title': "Weighted IRR (%)",
                'tickformat': '.1f',
                'ticksuffix': '%',
                'gridcolor': 'lightgrey',
                'zeroline': True,
            }
        })
        
        fig.update_layout(layout)
        return fig

    @staticmethod
    def create_dpi_chart(df: pd.DataFrame) -> go.Figure:
        df = df[df['Fund Type'].isin(STRATEGIES)].copy()
        
        dpi_data = df.groupby(['VY', 'Fund Type']).agg({
            'Capital Distributed': 'sum',
            'Capital Contributed': 'sum'
        }).reset_index()
        
        dpi_data['DPI'] = dpi_data['Capital Distributed'] / dpi_data['Capital Contributed']
        
        fig = go.Figure()
        
        for strategy in STRATEGIES:
            strategy_data = dpi_data[dpi_data['Fund Type'] == strategy].sort_values('VY')
            
            fig.add_trace(
                go.Scatter(
                    x=strategy_data['VY'],
                    y=strategy_data['DPI'],
                    name=strategy,
                    mode='lines+markers+text',
                    line=dict(color=PlotStyle.COLORS[strategy], width=2),
                    marker=dict(size=8),
                    text=[f"{val:.2f}x" for val in strategy_data['DPI']],
                    textposition="top center",
                    hovertemplate=(
                        "Year: %{x}<br>"
                        "DPI: %{y:.2f}x<br>"
                        "<extra></extra>"
                    )
                )
            )
        
        layout = PlotStyle.get_base_layout(title="DPI by Strategy and Vintage Year")
        layout.update({
            'xaxis': {
                'title': "Vintage Year",
                'dtick': 1,
                'gridcolor': 'lightgrey'
            },
            'yaxis': {
                'title': "DPI (Distributed to Paid-In Multiple)",
                'tickformat': '.2f',
                'ticksuffix': 'x',
                'gridcolor' : 'lightgrey',
                'rangemode': 'nonnegative'
            }
        })
        
        fig.update_layout(layout)
        return fig

    @staticmethod
    def create_irr_chart(df: pd.DataFrame) -> go.Figure:
        irr_data = []
        
        for strategy in STRATEGIES:
            strategy_data = df[df['Fund Type'] == strategy]
            
            for year, year_data in strategy_data.groupby('VY'):
                weighted_irr = DataProcessor.calculate_weighted_irr(year_data)
                irr_data.append({
                    'Year': year,
                    'Strategy': strategy,
                    'Weighted_IRR': weighted_irr
                })
        
        irr_df = pd.DataFrame(irr_data)
        fig = go.Figure()
    
        for strategy in STRATEGIES:
            strategy_data = irr_df[irr_df['Strategy'] == strategy].sort_values('Year')
            
            fig.add_trace(
                go.Scatter(
                    x=strategy_data['Year'],
                    y=strategy_data['Weighted_IRR'],
                    name=strategy,
                    mode='lines+markers+text',
                    line=dict(color=PlotStyle.COLORS[strategy], width=2),
                    marker=dict(size=8),
                    text=[f"{val:.1f}%" for val in strategy_data['Weighted_IRR']],
                    textposition="top center",
                    hovertemplate=(
                        "Year: %{x}<br>"
                        "Weighted IRR: %{y:.1f}%<br>"
                        "<extra></extra>"
                    )
                )
            )
        
        layout = PlotStyle.get_base_layout(title="Weighted IRR by Strategy and Vintage Year")
        layout.update({
            'xaxis': {
                'title': "Vintage Year",
                'dtick': 1,
                'gridcolor': 'lightgrey'
            },
            'yaxis': {
                'title': "Weighted IRR (%)",
                'tickformat': '.2f',
                'ticksuffix': '%',
                'gridcolor': 'lightgrey',
                'zeroline': True,
                'zerolinewidth': 1,
                'zerolinecolor': 'black'
            }
        })
        
        fig.update_layout(layout)
        return fig

In [151]:
df = pd.read_csv(CALSTRS_PATH)
df = DataProcessor.clean_data(df)

df.head()

,Description,VY,Capital Committed,Capital Contributed,Capital Distributed,Market Value,CalSTRS Since Inception IRR*,Fund Type,DPI
0,"Tenex Capital Partners III, L.P.",2021,100000000.0,98073921.0,66221945.0,93823287.0,99.80,Private Equity,0.675225
1,Spectrum VIII-A Discretionary Overage Program,2019,20000000.0,19321646.0,46332352.0,10737931.0,92.72,Growth Equity,2.397951
2,ICG Strategic Equity Fund V LP,2024,100000000.0,5400000.0,0.0,10129110.0,87.58,Private Equity,0.000000
3,ASF IX B L.P.,2022,250000000.0,26294373.0,821115.0,36606381.0,87.57,Private Equity,0.031228
4,"Francisco Partners Agility, L.P.",2017,50000000.0,44502467.0,171552467.0,56375649.0,84.71,Private Equity,3.854898


In [152]:
# Generate visualizations
viz_gen = VisualizationGenerator()
figures = {
    'strategy_distribution': viz_gen.create_strategy_pie_charts(df),
    'deployment': viz_gen.create_deployment_chart(df),
    'vc_deployment': viz_gen.create_venture_capital_deployment_chart(df),
    'capital_flows': viz_gen.create_capital_flows_chart(df),
    'vintage_irr': viz_gen.create_vintage_irr_chart(df),
    'strategy_irr': viz_gen.create_irr_chart(df),
    'strategy_dpi': viz_gen.create_dpi_chart(df)
}


In [117]:
figures["strategy_distribution"].show()

In [118]:
figures["deployment"].show()

In [156]:
figures["vc_deployment"].show()

In [133]:
figures["capital_flows"].show()

In [153]:
figures["vintage_irr"].show()

In [154]:
figures["strategy_dpi"].show()



In [155]:
figures["strategy_irr"].show()

In [59]:
mask = (df['VY'] >= 1998) & (df['VY'] <= 2013)
filtered_df = df[mask]

# Calculate totals 
total_distributed = filtered_df['Capital Distributed'].sum()
total_contributed = filtered_df['Capital Contributed'].sum()

# Print in billions and calculate DPI
print(f"Total Capital Distributed: ${total_distributed/1e9:.1f}B")
print(f"Total Capital Contributed: ${total_contributed/1e9:.1f}B")
print(f"Total DPI: {total_distributed/total_contributed:.2f}x")

Total Capital Distributed: $43.9B
Total Capital Contributed: $28.0B
Total DPI: 1.57x


In [62]:
mask = (df['VY'] > 2013) 
filtered_df = df[mask]

# Calculate totals 
total_distributed = filtered_df['Capital Distributed'].sum()
total_contributed = filtered_df['Capital Contributed'].sum()
total_market_value = filtered_df['Market Value'].sum()

# Print in billions and calculate DPI
print(f"Total Capital Distributed: ${total_distributed/1e9:.1f}B")
print(f"Total Capital Contributed: ${total_contributed/1e9:.1f}B")
print(f"Total DPI: {(total_distributed + total_market_value)/total_contributed:.2f}x")

Total Capital Distributed: $21.0B
Total Capital Contributed: $39.3B
Total DPI: 1.54x
